In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install pycountry-convert
!pip install country_converter --upgrade


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
import pycountry
import pycountry_convert as pc

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
covid_clean = pd.read_csv("../input/corona-virus-report/covid_19_clean_complete.csv",parse_dates=['Date'])

In [ ]:
covid_clean

In [ ]:
covid_clean.info()

In [ ]:
covid_clean.shape

In [ ]:
covid_clean.isna().sum()

In [ ]:
covid_clean['Active'] = covid_clean['Confirmed'] - covid_clean['Deaths'] - covid_clean['Recovered']

covid_clean['Country/Region'] = covid_clean['Country/Region'].replace('Mainland China', 'China')

covid_clean[['Province/State']] = covid_clean[['Province/State']].fillna('unknown')


In [ ]:
covid_clean.info()

In [ ]:
covid_clean["Country/Region"].unique()

In [ ]:
covid_ship =covid_clean[covid_clean['Province/State'].str.contains('Grand Princess')|covid_clean['Province/State'].str.contains('Diamond Princess cruise ship')]

In [ ]:
covid_clean.info()

In [ ]:
from functools import lru_cache
@lru_cache(maxsize=None)
def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
        return result[0].alpha_2
    except:
        return np.nan

covid_clean['country_code'] = covid_clean["Country/Region"].apply(lambda country: do_fuzzy_search(country))

In [ ]:
covid_clean.info()

In [ ]:
covid_clean["country_code"] = covid_clean.apply(lambda row:"TW" if row['Country/Region'] == 'Taipei and environs'   else row['country_code'],axis = 1)
covid_clean["country_code"] = covid_clean.apply(lambda row:"IR" if row['Country/Region'] == 'Iran (Islamic Republic of)' else row['country_code'],axis = 1) 
covid_clean["country_code"] = covid_clean.apply(lambda row:"PS" if row['Country/Region'] == 'occupied Palestinian territory' else row['country_code'],axis = 1)
covid_clean["country_code"] = covid_clean.apply(lambda row:"GB" if row['Country/Region'] == 'Channel Islands' else row['country_code'],axis = 1)
covid_clean["country_code"] = covid_clean.apply(lambda row:"TW" if row['Country/Region'] == 'Taiwan*'  else row['country_code'],axis = 1)
covid_clean["country_code"] = covid_clean.apply(lambda row:"KR" if row['Country/Region'] == 'Korea, South'  else row['country_code'],axis = 1)
covid_clean["country_code"] = covid_clean.apply(lambda row:"CD" if row['Country/Region'] == 'Congo (Kinshasa)'  else row['country_code'],axis = 1)


In [ ]:
@lru_cache(maxsize=None)
def do_continent_search(alpha2):
    try:
        result = pc.country_alpha2_to_continent_code(alpha2)
        return result
    except Exception:
        return np.nan



In [ ]:
covid_clean['continent'] = covid_clean["country_code"].apply(lambda x: do_continent_search(x))

In [ ]:
covid_clean["continent"] = covid_clean.apply(lambda row:"EU" if row['country_code'] == 'VA'   else row['continent'],axis = 1)


In [ ]:
covid_clean[covid_clean["continent"].isna()]


In [ ]:
import country_converter as coco
@lru_cache(maxsize=None)
def alpha3(alpha2):
    try:
        result =coco.convert(names=alpha2, to='ISO3',not_found = np.NaN)
        return result
    except Exception:
        return np.nan

In [ ]:
covid_clean['ISO3'] = covid_clean['country_code'].apply(lambda x:alpha3(x))

In [ ]:
covid_clean.info()

In [ ]:
px.line(covid_clean.groupby(['Date']).agg({'Confirmed':"sum"}).reset_index(),x = "Date",y = "Confirmed")

In [ ]:
temp = covid_clean.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

fig = px.area(temp, x="Date", y="Count", color='Case')
fig.show()

In [ ]:
px.line(covid_clean,x = "Date",y = "Confirmed")

In [ ]:
px.line(covid_clean.groupby(['Date','Country/Region']).agg({'Confirmed':"sum"}).reset_index(),x = "Date",y = "Confirmed",color = "Country/Region")

In [ ]:
latest_data = covid_clean[covid_clean["Date"] == max(covid_clean["Date"])].reset_index()

In [ ]:
latest_data

In [ ]:
latest_data.groupby('ISO3').sum().reset_index()

In [ ]:
fig = px.choropleth(latest_data.groupby('ISO3').sum().reset_index(), locations="ISO3", 
                    locationmode='ISO-3', color="Confirmed", 
                    hover_name="ISO3", range_color=[1,7000], 
                    title='Countries with Confirmed Cases',
                   color_continuous_scale=px.colors.sequential.Plasma)
fig.update(layout_coloraxis_showscale=True)
fig.show()

In [ ]:
fig = px.choropleth(latest_data.groupby('ISO3').sum().reset_index(), locations="ISO3", 
                    locationmode='ISO-3', color="Deaths", 
                    hover_name="ISO3", range_color=[1,7000], 
                    title='World Wide Covid Deaths',
                   color_continuous_scale=px.colors.sequential.Plasma)
fig.update(layout_coloraxis_showscale=True)
fig.show()

In [ ]:
fig = px.choropleth(latest_data.groupby('ISO3').sum().reset_index(), locations="ISO3", 
                    locationmode='ISO-3', color="Recovered", 
                    hover_name="ISO3", range_color=[1,7000], 
                    title='Recovered Cases World Wide ',
                   color_continuous_scale=px.colors.sequential.Plasma)
fig.update(layout_coloraxis_showscale=True)
fig.show()

In [ ]:
country_latest = latest_data.groupby("ISO3").sum().reset_index()
country_latest

In [ ]:
country_latest["Mortality_Rate"] = 100*country_latest["Deaths"]/country_latest["Confirmed"]
country_latest

In [ ]:
mortality = country_latest[country_latest["Mortality_Rate"]>0].sort_values(by = 'Mortality_Rate',ascending= False)


In [ ]:
plt.figure(figsize = (16,10))
sns.barplot(y  = mortality.Mortality_Rate , x = mortality.ISO3,orient = 'v',)
plt.show()

MAR: Morocco

BGR: Bulgaria

PAN: Panama

IRQ:IRAQ

ALB:Albania

